In [3]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image

# ✅ Define the model class (same as in training)
class ModifiedResNeXt(nn.Module):
    def __init__(self):
        super(ModifiedResNeXt, self).__init__()
        resnext = models.resnext50_32x4d(weights=models.ResNeXt50_32X4D_Weights.IMAGENET1K_V1)
        self.features = nn.Sequential(*list(resnext.children())[:-1])
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.BatchNorm1d(2048),
            nn.Dropout(0.5),
            nn.Linear(2048, 512),
            nn.ReLU(),
            nn.BatchNorm1d(512),
            nn.Dropout(0.3),
            nn.Linear(512, 2)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

# ✅ Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ✅ Load the model
model = ModifiedResNeXt()
model.load_state_dict(torch.load('best_resnext50_model-2.pth', map_location=device))
model.to(device)
model.eval()

# ✅ Define transformation (same as test/val transform)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# ✅ Load and preprocess a single image
def predict_image(image_path):
    image = Image.open(image_path).convert("RGB")
    image = transform(image).unsqueeze(0).to(device)  # Add batch dimension
    with torch.no_grad():
        outputs = model(image)
        _, predicted = torch.max(outputs, 1)
        confidence = torch.softmax(outputs, dim=1)[0][predicted].item()
    label = predicted.item()
    return label, confidence

# ✅ Run inference
img_path = r"C:\Users\rocky's pc\OneDrive\Desktop\visualization_gossipcop-928457.png"  # 🔁 Replace with actual image path
label_map = {0: "REAL", 1: "FAKE"}         # 🔁 Adjust if your label mapping differs

predicted_label, conf = predict_image(img_path)
print(f"🧠 Prediction: {label_map[predicted_label]} ({conf*100:.2f}% confidence)")


🧠 Prediction: FAKE (93.48% confidence)
